# Week 5 - a : Generate Lyrics with charRNN
1. 한국 노래 가사 데이터로 훈련셋 구축하기
2. Sequential Dataset을 여러개의 windows로 나누기
3. Building & Training the CharRNN model
4. Play!

In [1]:
# 이번에는 tensorflow를 사용해봅시다!
import tensorflow as tf  # ready-made RNN을 사용하기 위해!
from tensorflow import keras  # tensorflow 안에 있는 keras 라이브러리를 사용!
import numpy as np
import requests
import os
import time

In [2]:
# gpu 사용가능 여부 체크
# 출처: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## 1. 훈련셋 구축하기

한국어 가사 데이터셋을 다운로드 하여, RNN학습을 위한 데이터 셋을 구축해봅시다.


In [3]:
# 한국어 발라드 가사 데이터를 불러오기.
LYRICS_URL: str = 'https://raw.githubusercontent.com/ahastudio/ballad-lyrics-maker/master/data/lyrics_ballad/input.txt'
corpus = requests.get(LYRICS_URL).text

In [4]:
# 첫 200글자 데이터 확인.
#학습 하고자 하는 데이터는 charRNN-> 단어가 아니라 글자가 토큰이다.
#corpus : str
print(corpus[:1000])

내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데 음~오 
제발 내 곁에서 떠나가지 말아요
그대없는 밤은 너무 싫어
우~우~우~ 돌이킬수 없는 그대 마음 
우~우~우~ 이제와서 다시 어쩌려나
슬픔마음도 이젠 소용없네 

내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데 음~오 
제발 내 곁에서 떠나가지 말아요
그대없는 밤은 너무 싫어
우~우~우~ 돌이킬수 없는 그대 마음 
우~우~우~ 이제와서 다시 어쩌려나
슬픔마음도 이젠 


조용한 밤하늘에
아름다운 별빛이 
멀리 있는 창가에도
소리 없이 비추고 
한낮의 기억들은
어디론가 사라져 
꿈을 꾸는 저 하늘만
바라보고 있어요 
부드러운 노래 소리에
내 마음은 아이처럼 
파란 추억의 바다로 
뛰어가고 있네요 
깊은 밤 아름다운 그 시간은 
이렇게 찾아와 마음을 물들이고 
영원한 여름밤의 꿈을
기억하고 있어요 
다시 아침이 밝아와도
잊혀지지 않도록
부드러운 노래 소리에
내 마음은 아이처럼 
파란 추억의 바다로 
뛰어가고 있네요 
깊은 밤 아름다운 그 시간은 
이렇게 찾아와 마음을 물들이고 
영원한 여름밤의 꿈을
기억하고 있어요 
다시 아침이 밝아와도
잊혀지지 않도록

나의 하늘을 본 적이 있을까
조각 구름과 빛나는 별들이
끝없이 펼쳐 있는
구석진 그 하늘 어디선가
내 노래는 널 부르고 있음을
너는 듣고 있는지
나의 정원을 본 적이 있을까
국화와 장미 예쁜 사루비아가
끝없이 피어 있는
언제든 그 문은 열려 있고
그 향기는 널 부르고 있음을
넌 알고 있는지
나의 어릴 적 내 꿈만큼이나
아름다운 가을 하늘이랑
네가 그것들과 손잡고
고요한 달빛으로 내게 오면
내 여린 마음으로 피워낸
나의 사랑을
너에게 꺾어줄게
나의 어릴 적 내 꿈만큼이나
아름다운 가을 하늘이랑
네가 

In [5]:
# 다음으로는, 먼저 각 글자별 정수 인코딩을 진행해주어야합니다.
# char_level = True로 설정해주면, 단어가 아닌 글자 단위로 정수 인코딩을 해줍니다.
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
# 말뭉치에 토크나이저를 학습시킵니다. 각 글자에 대응하는 정수 인코딩을 찾아줍니다!
tokenizer.fit_on_texts([corpus])

In [6]:
# 한번 예시를 살펴볼까요?
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences
print(tokenizer.texts_to_sequences(["난 너를 사랑해"]))
print(tokenizer.sequences_to_texts([[59, 1, 38, 35, 1, 21, 27, 28]]))

[[59, 1, 38, 35, 1, 21, 27, 28]]
['난   너 를   사 랑 해']


In [7]:
# 고유한 글자의 개수는?
vocab_size = len(tokenizer.word_index)
print(vocab_size)
# 전체 데이터셋의 크기는?
dataset_size = sum([
                count
                # word_counts: Dict[Tuple[str=글자, int=빈도수]]
                # -> items -> List[Tuple[str, count]]
                # -> loop -> str, int -> int
                for character, count in tokenizer.word_counts.items()
])
print(dataset_size)

1207
742966


In [8]:
# 텍스트 데이터 -> 정수 인코딩 데이터로 전처리하기
[corpus_encoded] = np.array(tokenizer.texts_to_sequences(texts=[corpus]))

In [27]:
print(corpus[:100])
print(corpus_encoded[:100])

내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데
[ 19   1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17
  34   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1
 120   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51
  49  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514
   1 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7
   1  19 312   1 161  89 306  73   5 101]


### 학습에 필요한 데이터 구축하기


In [10]:
# 전체 데이터셋의 90%만 훈련셋으로 사용
train_size = dataset_size * 70 // 100
# 리스트 슬라이싱으로 train & test 구분 
# Dataset.from_tensor_slices: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices
ds = tf.data.Dataset.from_tensor_slices(corpus_encoded[:train_size])

In [11]:
# ds: Dataset[Tensor]
# 각 sample은 하나의 숫자를 담고 있는 Tensor
# ds.take: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take
for sample in ds.take(3):
  print(sample)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(158, shape=(), dtype=int64)


### To-do 1 
다음의 질문에 답하세요:

시계열 데이터를 훈련-테스트 셋으로 분리할 때, 단순히 앞 70%를 훈련셋으로, 뒤 30%를 훈련 셋으로 사용하는 경우, 어떤 문제가 발생할 수 있을까요? (hint: 과거에 있었던 일이 미래에도 일어날 것이라고 가정할 수 있을까요?)

---
답:시계열 데이터는 수학적인 규칙이 없다 
시간에 따른 데이터는 달라질수 있다? 
자료에서 본 날씨 에 따른 어떤 음식을 먹어야 하는지에 대한 예측?

그럼 어떻게 학습 하냐?  구간을 나눠서 구간별로 train test split 을 한다

train
[2020-2019]*0.7 
[2019-2018]*0.7 
[2018-2017]*0.7
test
[2020-2019]*0.3 
[2019-2018]*0.3 
[2018-2017]*0.3
---


## 2. Sequential Dataset을 여러개의 windows로 나누기

In [12]:
WINDOW_SIZE = 100 
# Dataset.window(): https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices
# target = input shifted 1 character ahead. (그래서 shift=1)로 설정함.
ds = ds.window(WINDOW_SIZE, shift=1, drop_remainder=True)

In [13]:
# ds: Dataset[Dataset]
# 완성된 윈도우 확인 - 하나씩 shift되어 있는 것을 볼 수 있음!
for sample in ds.take(3):
  print(list(sample.as_numpy_iterator()))

[19, 1, 158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 19, 7, 1, 514, 1, 104, 43, 1, 33, 12, 63, 1, 81, 107, 12, 1, 28, 176, 2, 4, 17, 7, 1, 19, 312, 1, 161, 89, 306, 73, 5, 101]
[1, 158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 19, 7, 1, 514, 1, 104, 43, 1, 33, 12, 63, 1, 81, 107, 12, 1, 28, 176, 2, 4, 17, 7, 1, 19, 312, 1, 161, 89, 306, 73, 5, 101, 1]
[158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 1

In [14]:
# Dataset 안에 또다른 Dataset이 있다. 
# -> Dataset 속에 Tensor가 있도록 바꿔주자!
ds = ds.flat_map(lambda window: window.batch(WINDOW_SIZE)) #batch 를 사용하면 길이 에 맞는 tensor 가 만들어진다. batch -> loss 계산하기 위한 작은 단위 

In [15]:
# ds: Dataset[Tensor] 확인하기!
for sample in ds.take(3):
  print(sample)

tf.Tensor(
[ 19   1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17
  34   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1
 120   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51
  49  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514
   1 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7
   1  19 312   1 161  89 306  73   5 101], shape=(100,), dtype=int64)
tf.Tensor(
[  1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17  34
   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1 120
   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51  49
  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514   1
 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7   1
  19 312   1 161  89 306  73   5 101   1], shape=(100,), dtype=int64)
tf.Tensor(
[158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17  34   5
   1 115  18   1  38  51

In [16]:
# 로스를 구하는 단위 = batch
# 로스를 구할 때, 32개의 window를 대상으로 로스를 계산하자.
BATCH_SIZE = 32
ds = ds.shuffle(10000).batch(BATCH_SIZE)

In [17]:
for batch in ds.take(3):
  # 하나의 배치 = 32개의 윈도우가 들어있고, 각 윈도우의 크기는 100
  print(batch.shape)

(32, 100)
(32, 100)
(32, 100)


In [18]:
# x & y 만들기 (입력 & label)
# e.g. 
# input = 난,너,를,사,랑
# target =너,를,사,랑,해
#입력 과 출력이 바로 나열이다 
#즉, P(너|난), P(를|난,너), P(사|난,너,를), P(랑|난,너,를,사), P(해|난,너,를,사,랑)를 최대화 하도록 RNN의 가중치를 학습시키고자 하는 것.
ds = ds.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

In [19]:
for batch in ds.take(1):
  # 하나의 배치 = 32개의 윈도우가 들어있고, 각 윈도우의 크기는 100
  X, Y = batch
  print(X[0])
  print("---")
  print(Y[0])

tf.Tensor(
[ 22   2   6  24   1   9 215   1 230   1  19   1 133  26 359   3   6   2
  10  98  14  68   1   7  12   1  13  66   3  27   2  44   7   1   4  75
  37 189   1 184 211  15   2  15  25  30   1 168 149  52  23   1  19  22
   1  42  31   2  19   1  56 110   1  39  40  52  23   1 250  93 482   2
   6  24   1  21  27  12   2  38  11  22   1 631   9 164  22   2   2  21
  27  18   1   4 100  22   1  90  15], shape=(99,), dtype=int64)
---
tf.Tensor(
[  2   6  24   1   9 215   1 230   1  19   1 133  26 359   3   6   2  10
  98  14  68   1   7  12   1  13  66   3  27   2  44   7   1   4  75  37
 189   1 184 211  15   2  15  25  30   1 168 149  52  23   1  19  22   1
  42  31   2  19   1  56 110   1  39  40  52  23   1 250  93 482   2   6
  24   1  21  27  12   2  38  11  22   1 631   9 164  22   2   2  21  27
  18   1   4 100  22   1  90  15   1], shape=(99,), dtype=int64)


In [30]:
# prefetch: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take
# 더 빠른 훈련을 위해 필요. 하나의 캐릭터를 처리할 때, 미리 다음 캐릭터를 로드해두도록 할 수 있음.
#책을 한권씩 빌리는게 아니라 한꺼번에 빌리는 느낌
ds = ds.prefetch(1)

## 3. Building & Training the CharRNN model

In [21]:
EMBEDDING_DIM = 100
RNN_UNITS = 1024
def load_rnn():
  model = tf.keras.Sequential([
    # one-hot encoding 대신, embedding 레이어를 사용!                               
    tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM,
                              batch_input_shape=[BATCH_SIZE, None]),
    # 간단한 RNN 레이어 하나.                               
    tf.keras.layers.SimpleRNN(RNN_UNITS,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    # 마지막은 모든 글자에 대한 점수를 출력하는 linear layer
    tf.keras.layers.Dense(vocab_size)])
  return model
model = load_rnn()
# 분류문제를 푸는 것이므로, cross entropy loss를 사용한다.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 100)           120700    
_________________________________________________________________
simple_rnn (SimpleRNN)       (32, None, 1024)          1152000   
_________________________________________________________________
dense (Dense)                (32, None, 1207)          1237175   
Total params: 2,509,875
Trainable params: 2,509,875
Non-trainable params: 0
_________________________________________________________________


### To-do 2
다음의 질문에 답하세요:

입력값을 원-핫 인코딩을 하게될 경우, 위처럼 굳이 임베딩 레이어를 추가할 필요는 없습니다. 하지만 우리는 원핫 인코딩보다는 임베딩 레이어를 추가하는 것이 더 적절한 상황입니다. 그 이유는 무엇일까요?
( hint: 고유한 글자가 몇개있나요?)

---
답: one hot vector 를 사용하면 하나의 원소는 1이고 나머지 원소는 0 이 되는 구조를 갇게 된다. 이거는 단어간 특정한 의미를 갖는것이 아니고 독립적인 의미를 갖게 된다.  -> RNN 을 구현하는 구조에서는 맞지 않는다?

임베딩 레이어는 독립적인 구조가 아닌 이는 비슷한 분포를 가진 단어의 주변 단어들은 비슷한 의미를 가진다는 것을 말합니다.

word 를 미리 정의된 차원에서 연속형의 값을 갖는 벡터로 표현됩니다.
word 를 연속형의 벡터로 만들어준다?

---

In [22]:
# 체크포인트가 저장될 디렉토리
# 이걸 현재까지 가장 나은 모델을 알아서 저장해준다.
CHECKPOINT_PATH = './models/my_checkpt.ckpt'
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    save_weights_only=True, 
    save_best_only=True,
    monitor='loss', 
    verbose=1)


In [23]:
# 캐릭터 분류문제이므로, categorical cross entropy loss를 사용 
# GPU를 사용해도, 훈련에 상당한 시간이 걸릴 것.
# 그래서 epoch은 3으로만 설정하겠다!
EPOCHS = 10
STEPS_PER_EPOCH = 172
history = model.fit(ds,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 28s 134ms/step - loss: 13.3598 - acc: 0.0112

Epoch 00001: loss improved from inf to 13.35983, saving model to ./models/my_checkpt.ckpt
Epoch 2/10
172/172 [==============================] - 23s 134ms/step - loss: 13.5191 - acc: 3.1566e-04

Epoch 00002: loss did not improve from 13.35983
Epoch 3/10
172/172 [==============================] - 23s 134ms/step - loss: 13.5848 - acc: 0.0012

Epoch 00003: loss did not improve from 13.35983
Epoch 4/10
172/172 [==============================] - 23s 134ms/step - loss: 13.7122 - acc: 0.0018

Epoch 00004: loss did not improve from 13.35983
Epoch 5/10
172/172 [==============================] - 23s 136ms/step - loss: 13.7368 - acc: 3.4135e-04

Epoch 00005: loss did not improve from 13.35983
Epoch 6/10
172/172 [==============================] - 23s 134ms/step - loss: 13.7772 - acc: 3.1199e-05

Epoch 00006: loss did not improve from 13.35983
Epoch 7/10
172/172 [==============================] - 23s 

In [24]:
# 가장 성능이 좋은 모델을 로드
model = load_rnn()
model.load_weights(CHECKPOINT_PATH)
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (32, None, 100)           120700    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (32, None, 1024)          1152000   
_________________________________________________________________
dense_1 (Dense)              (32, None, 1207)          1237175   
Total params: 2,509,875
Trainable params: 2,509,875
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_text(model, start_string: str, temp: float, num_chars: int):
    global tokenizer
    # --- temp --- #
    # 온도가 높으면 더 의외의 텍스트가 됩니다.
    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)
    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = tokenizer.texts_to_sequences(start_string)
    input_eval = tf.expand_dims(input_eval, 0)
    # 결과를 저장할 빈 문자열
    text_generated = []
    # 최적의 세팅을 찾기 위한 실험
    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_chars):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)
        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # 예측된 단어를 다음 입력으로 모델에 전달
        input_eval = tf.expand_dims([predicted_id], 0)
        decoded = tokenizer.index_word[predicted_id]
        print(decoded)
        text_generated.append(decoded)
    return (start_string + ''.join(text_generated))

## 4. Play!

In [28]:
#print(generate_text(model, start_string=u"그대를 만나고\n", temp=0.8, num_chars=100))

In [29]:
print(generate_text(model, start_string=u"그리워하면 언젠간 만나게 되는", temp=0.8, num_chars=100))

ValueError: ignored

In [ ]:
print(generate_text(model, start_string=u"또 하루 멀어져", temp=0.8, num_chars=100))

In [ ]:
# 기훈님
print(generate_text(model, start_string="꽃이 예뻐봤자 뭐해\n", temp=0.8, num_chars=100))

In [ ]:
# 경서님
print(generate_text(model, start_string="피노키오\n", temp=0.8, num_chars=100))

In [ ]:
# 경서님
print(generate_text(model, start_string="피노키오 ", temp=0.8, num_chars=100))

In [ ]:
# 선영님
print(generate_text(model, start_string="광야로 걸어가 ", temp=0.8, num_chars=100))

In [ ]:
# 찬우님
print(generate_text(model, start_string="아 뜨거워 아 뜨거워 주님의 사랑\n", temp=0.8, num_chars=100))